<font>
<div dir=ltr align=center>
<img src='Sharif_logo.png' width=250 height=250> <br>
<font color=0F5298 size=7>
Applied Data Science<br>
<font color=2565AE size=5>
Spring 2025<br>
<font color=3C99D size=5>
HW4 - Feature Engineering <br>
<font color=696880 size=4>
Ali Mohammadzade Shabestari - 401106482 - Computer Engineering



# 1. Import Libraries

In [157]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest, mutual_info_regression
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# 2. Load Datasets

In the previous notebook, I cleaned Orders and Products datasets. I saved them in `Digikala Dataset` as `dff_orders.csv` and `dff_product.csv` folder and now I upload them to visualize data and work on their features.

In [158]:
dff_orders = pd.read_csv('../Digikala Dataset/dff_orders.csv')
dff_product = pd.read_csv('../Digikala Dataset/dff_product.csv')

dff_orders['DateTime_CartFinalize'] = pd.to_datetime(dff_orders['DateTime_CartFinalize'])

# 3. Creating New Features

First, let's inspect top of dataframe.

In [159]:
dff_orders.head()

,ID_Order,ID_Customer,ID_Item,DateTime_CartFinalize,Amount_Gross_Order,city,Quantity_item,province,year,date
0,2714054,469662,21386,2015-10-15 08:50:56,597982.0,Mahmudabad,1.0,Mazandaran,2015,2015-10-15
1,11104039,3063877,248497,2018-02-11 00:29:26,980000.0,Khorramdarreh,1.0,Zanjan,2018,2018-02-11
2,4228130,3184893,50144,2016-06-14 00:30:08,229358.0,Qarchak,1.0,Tehran,2016,2016-06-14
3,22225624,6888562,70208,2018-09-03 14:37:19,16514.0,Qom,1.0,Qom,2018,2018-09-03
4,4068771,2533490,67627,2016-05-21 11:51:02,133028.0,Tehran,1.0,Tehran,2016,2016-05-21


## 3.1. Ratio

To create a ratio feature, you can divide one numerical feature by another. For example, you can create a new feature that represents the price per item.

In [160]:
# Create a new feature 'Price_per_Quantity' by dividing 'Amount_Gross_Order' by 'Quantity_item'
dff_orders['Price_per_Quantity'] = dff_orders['Amount_Gross_Order'] / dff_orders['Quantity_item']

# Display few rows with Quantity > 1 to verify the new feature
dff_orders[dff_orders['Quantity_item'] > 1].head()

,ID_Order,ID_Customer,ID_Item,DateTime_CartFinalize,Amount_Gross_Order,city,Quantity_item,province,year,date,Price_per_Quantity
16,23062506,7193109,1656648,2018-10-04 17:48:12,229000.0,Tehran,2.0,Tehran,2018,2018-10-04,114500.000000
18,23693949,3665022,1671745,2018-10-30 22:15:15,73211.0,Karaj,3.0,Alborz,2018,2018-10-30,24403.666667
20,6156363,3549803,193926,2017-01-24 03:05:35,41284.0,Tehran,3.0,Tehran,2017,2017-01-24,13761.333333
44,23634499,5196834,1157587,2018-10-28 16:07:17,142982.0,Tehran,3.0,Tehran,2018,2018-10-28,47660.666667
50,8186946,5356355,395337,2017-08-12 11:10:35,1950000.0,Unknown,2.0,NaN,2017,2017-08-12,975000.000000


## 3.2. Binning

Binning is a technique used to categorize continuous variables into discrete bins or intervals. This can be useful for simplifying the analysis and visualization of data.

 In the context of our dataset, we are binning the `total_price` into four equal-width bins. This means that the range of `total_price` values will be divided into four intervals, and each order will be assigned a label ('Low', 'Medium', 'High', 'Very High') based on which interval its `total_price` falls into. This can help us to easily identify and analyze orders based on their price categories.

In [161]:
# Binning 'total_price' into 4 equal-width bins
dff_orders['Price_Bin'] = pd.cut(dff_orders['Amount_Gross_Order'], bins=4, labels=['Low', 'Medium', 'High', 'Very High'])
print(dff_orders['Price_Bin'].value_counts())

Price_Bin
Low          199999
Very High         1
Medium            0
High              0
Name: count, dtype: int64


Because of a high outlier, it's better to ignore them while binning.

In [162]:
# Quantile-based binning of 'total_price' into 4 bins
dff_orders['Price_Bin'] = pd.qcut(dff_orders['Amount_Gross_Order'], q=4, labels=['Low', 'Medium', 'High', 'Very High'])
print(dff_orders['Price_Bin'].value_counts())

Price_Bin
Medium       50516
Low          50000
Very High    50000
High         49484
Name: count, dtype: int64


## 3. 3. Function of A Column

A great example for function of a column is province mapping that has been done before.

As another exmaple, you can apply logarithmic transformation on price.

In [163]:
# Example 1: Log transformation of 'total_price' (Adding 1 to avoid log(0))
dff_orders['Log_Total_Price'] = np.log(dff_orders['Amount_Gross_Order'] + 1)
dff_orders.head(5)

,ID_Order,ID_Customer,ID_Item,DateTime_CartFinalize,Amount_Gross_Order,city,Quantity_item,province,year,date,Price_per_Quantity,Price_Bin,Log_Total_Price
0,2714054,469662,21386,2015-10-15 08:50:56,597982.0,Mahmudabad,1.0,Mazandaran,2015,2015-10-15,597982.0,High,13.301318
1,11104039,3063877,248497,2018-02-11 00:29:26,980000.0,Khorramdarreh,1.0,Zanjan,2018,2018-02-11,980000.0,High,13.795309
2,4228130,3184893,50144,2016-06-14 00:30:08,229358.0,Qarchak,1.0,Tehran,2016,2016-06-14,229358.0,Medium,12.343044
3,22225624,6888562,70208,2018-09-03 14:37:19,16514.0,Qom,1.0,Qom,2018,2018-09-03,16514.0,Low,9.712024
4,4068771,2533490,67627,2016-05-21 11:51:02,133028.0,Tehran,1.0,Tehran,2016,2016-05-21,133028.0,Medium,11.798322


## 3. 4. Combining Columns

In this dataset, a few meaningful features can be created by combining features. 

In [164]:
dff_orders["Region_ID"] = dff_orders["city"] + "_" + dff_orders["province"]
dff_orders.head()

,ID_Order,ID_Customer,ID_Item,DateTime_CartFinalize,Amount_Gross_Order,city,Quantity_item,province,year,date,Price_per_Quantity,Price_Bin,Log_Total_Price,Region_ID
0,2714054,469662,21386,2015-10-15 08:50:56,597982.0,Mahmudabad,1.0,Mazandaran,2015,2015-10-15,597982.0,High,13.301318,Mahmudabad_Mazandaran
1,11104039,3063877,248497,2018-02-11 00:29:26,980000.0,Khorramdarreh,1.0,Zanjan,2018,2018-02-11,980000.0,High,13.795309,Khorramdarreh_Zanjan
2,4228130,3184893,50144,2016-06-14 00:30:08,229358.0,Qarchak,1.0,Tehran,2016,2016-06-14,229358.0,Medium,12.343044,Qarchak_Tehran
3,22225624,6888562,70208,2018-09-03 14:37:19,16514.0,Qom,1.0,Qom,2018,2018-09-03,16514.0,Low,9.712024,Qom_Qom
4,4068771,2533490,67627,2016-05-21 11:51:02,133028.0,Tehran,1.0,Tehran,2016,2016-05-21,133028.0,Medium,11.798322,Tehran_Tehran


## 3. 5. Date/Time

There are many options for date/time extraction including: hour, day, month and ... . But I guess weekday and is_weekend are more useful.

In [165]:
dff_orders["Weekday"] = dff_orders["DateTime_CartFinalize"].dt.dayofweek  # 0=Monday, 6=Sunday
dff_orders["Is_Weekend"] = dff_orders["Weekday"].isin([5, 6]).astype(int)  # Binary weekend flag
dff_orders.head()

,ID_Order,ID_Customer,ID_Item,DateTime_CartFinalize,Amount_Gross_Order,city,Quantity_item,province,year,date,Price_per_Quantity,Price_Bin,Log_Total_Price,Region_ID,Weekday,Is_Weekend
0,2714054,469662,21386,2015-10-15 08:50:56,597982.0,Mahmudabad,1.0,Mazandaran,2015,2015-10-15,597982.0,High,13.301318,Mahmudabad_Mazandaran,3,0
1,11104039,3063877,248497,2018-02-11 00:29:26,980000.0,Khorramdarreh,1.0,Zanjan,2018,2018-02-11,980000.0,High,13.795309,Khorramdarreh_Zanjan,6,1
2,4228130,3184893,50144,2016-06-14 00:30:08,229358.0,Qarchak,1.0,Tehran,2016,2016-06-14,229358.0,Medium,12.343044,Qarchak_Tehran,1,0
3,22225624,6888562,70208,2018-09-03 14:37:19,16514.0,Qom,1.0,Qom,2018,2018-09-03,16514.0,Low,9.712024,Qom_Qom,0,0
4,4068771,2533490,67627,2016-05-21 11:51:02,133028.0,Tehran,1.0,Tehran,2016,2016-05-21,133028.0,Medium,11.798322,Tehran_Tehran,5,1


## 3. 6. Count & Aggregation

Aggregating data helps uncover trends and behaviors at different levels, such as customer behavior, order frequency, and regional sales trends.

In [166]:
# Create a new feature 'Customer_Total_Orders' which counts the total number of orders for each customer
dff_orders["Customer_Total_Orders"] = dff_orders.groupby("ID_Customer")["ID_Order"].transform("count")

# Create a new feature 'City_Avg_Order_Value' which calculates the average order value for each city
dff_orders["City_Avg_Order_Value"] = dff_orders.groupby("city")["Amount_Gross_Order"].transform("mean")

dff_orders.head()

,ID_Order,ID_Customer,ID_Item,DateTime_CartFinalize,Amount_Gross_Order,city,Quantity_item,province,year,date,Price_per_Quantity,Price_Bin,Log_Total_Price,Region_ID,Weekday,Is_Weekend,Customer_Total_Orders,City_Avg_Order_Value
0,2714054,469662,21386,2015-10-15 08:50:56,597982.0,Mahmudabad,1.0,Mazandaran,2015,2015-10-15,597982.0,High,13.301318,Mahmudabad_Mazandaran,3,0,6,1.664577e+06
1,11104039,3063877,248497,2018-02-11 00:29:26,980000.0,Khorramdarreh,1.0,Zanjan,2018,2018-02-11,980000.0,High,13.795309,Khorramdarreh_Zanjan,6,1,1,7.466292e+05
2,4228130,3184893,50144,2016-06-14 00:30:08,229358.0,Qarchak,1.0,Tehran,2016,2016-06-14,229358.0,Medium,12.343044,Qarchak_Tehran,1,0,3,1.049428e+06
3,22225624,6888562,70208,2018-09-03 14:37:19,16514.0,Qom,1.0,Qom,2018,2018-09-03,16514.0,Low,9.712024,Qom_Qom,0,0,1,1.448778e+06
4,4068771,2533490,67627,2016-05-21 11:51:02,133028.0,Tehran,1.0,Tehran,2016,2016-05-21,133028.0,Medium,11.798322,Tehran_Tehran,5,1,1,1.410293e+06


# 4. Feature Selection Based on Mutual Information

Mutual information is a measure of the dependency between variables. It quantifies the amount of information obtained about one variable through the other variable. In the context of feature selection, mutual information helps identify the features that have the highest dependency with the target variable, thus indicating their importance.

For mutual information to be calculated, the features need to be numeric. This is because mutual information algorithms typically require numerical input to compute the dependency measures. Therefore, categorical features, such as province, need to be encoded into numeric values. This can be achieved using techniques like label encoding, which assigns a unique integer to each category. This transformation allows the mutual information calculation to proceed correctly. We also need to drop columns that are most likely not useful for our model.

In [167]:
# Transform 'Price_Bin' into numeric values
def convert_price_bin_to_numeric(df, column_name='Price_Bin'):
    bin_mapping = {'Low': 1, 'Medium': 2, 'High': 3, 'Very High': 4}
    df[column_name + '_Numeric'] = df[column_name].map(bin_mapping)
    return df

dff_orders = convert_price_bin_to_numeric(dff_orders)

# Encode the 'province' column
le = LabelEncoder()
dff_orders['province_encoded'] = le.fit_transform(dff_orders['province'])

In [168]:
# Select the features and target variable
X = dff_orders.drop(columns=['ID_Order', 'ID_Customer', 'ID_Item', 'DateTime_CartFinalize', 'city', 'date', 'Region_ID', 'Price_Bin', 'province', 'Log_Total_Price', 'Amount_Gross_Order', 'Price_per_Quantity'])

y = dff_orders['Price_Bin_Numeric']

# Apply SelectKBest with mutual_info_regression
selector = SelectKBest(score_func=mutual_info_regression, k='all')
selector.fit(X, y)

# Get the scores
scores = selector.scores_

# Create a DataFrame to display the mutual information scores
mi_df = pd.DataFrame({'Feature': X.columns, 'Mutual Information': scores})
mi_df = mi_df.sort_values(by='Mutual Information', ascending=False)

print(mi_df)

                 Feature  Mutual Information
6      Price_Bin_Numeric            1.386522
0          Quantity_item            0.016793
4  Customer_Total_Orders            0.013572
1                   year            0.006540
5   City_Avg_Order_Value            0.003032
2                Weekday            0.000000
3             Is_Weekend            0.000000
7       province_encoded            0.000000


Features with higher mutual information scores are more relevant for predicting the target variable. These features should be prioritized in the modeling process, as they provide more information about the target variable.


# 5. Dimensionality Reduction Using PCA

I asked ChatGPT about what columns are important for PCA and it said:

PCA only works on numerical columns, so:

✅ Include:

- Continuous numerical features (e.g., price, age, weight).
- Features that are on a comparable scale (or will be standardized).

❌ Exclude or Preprocess:

- Categorical features (convert them first using One-Hot Encoding or Label Encoding).
- ID columns (e.g., Customer ID, Order ID – they don’t hold meaningful variance).
- Date/Time columns (unless converted into numerical values like hours, weekdays, etc.).

Therefore, we use X as the same as before and apply PCA on it.

In [169]:
# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Apply PCA
pca = PCA(n_components=2)  # Adjust the number of components as needed
X_pca = pca.fit_transform(X_scaled)

# Create a DataFrame to display the PCA results
pca_df = pd.DataFrame(data=X_pca, columns=['Principal Component 1', 'Principal Component 2'])
pca_df['Price_Bin_Numeric'] = y.values

print(pca_df.head())

   Principal Component 1  Principal Component 2 Price_Bin_Numeric
0              -0.452267               0.463341                 3
1               2.151072              -1.793826                 3
2              -1.130402              -1.552247                 2
3              -1.476265              -0.428096                 1
4               1.793088              -0.713397                 2


# 6. Answer A Question!

**Question:**

When is feature engineering a “nice to have option” and in what situations it is a “must to have”?



**Answer:**

Feature engineering can be a "nice to have option" when the dataset is already well-structured, contains relevant features, and the machine learning model can achieve good performance with minimal feature engineering. It is useful for quickly prototyping and testing models. However, feature engineering becomes a "must to have" when the dataset contains raw or unstructured data that needs transformation into meaningful features, the existing features are insufficient to capture underlying patterns, or the problem is complex and requires additional features to improve model performance. It is essential when the model's performance is unsatisfactory, and feature engineering can help extract more relevant information from the data.

✅ In this project, we performed various feature engineering techniques, but imo it is a 'Must To Have' option since we obtained important features like Province, Price_Bin, Log_Total_Price, Region_ID, Weekday, Is_Weekend, and ... . These features can help a Machine Learning model to predict the price category of an order more accurately.


